# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise preditiva

### 1. Importando pacotes de análise

In [ ]:
# IMPORTANDO PACOTES 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd  
import chart_studio.plotly as py
import plotly.graph_objs as gp 
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split

### 2. Importando dados

In [ ]:
#IMPOTANDO DADOS
sih_fn = pd.read_csv("sih_pneumonia_1721_trat.csv", index_col= 'N_AIH')
sih_fn.info()

### 3. Definindo teste e treino

In [ ]:
#SEPARANDO OS DADOS
train, test = train_test_split(sih_fn, train_size=0.70, random_state=42)

### 4. Testando algoritmos de classificação

#### 4.1 Regressão Logística

In [ ]:
log_reg = smf.logit("MORTE ~ IDADE + SEXO + DIAS_PERM + DIAG_PRINC + MARCA_UTI + UTI_MES_TO + REGIAO", data = train).fit()
print(log_reg.summary())